In [1]:
# ============================================================
# 0) IMPORTS & CONFIG
# ============================================================
import dspy
import torch
import re
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# Base model for reasoning
lm = dspy.LM(model="ollama_chat/gpt-oss:20b", api_base="http://localhost:11434", api_key="", temperature=1.0)
dspy.configure(lm=lm)

# Stronger LM for reflection
reflection_lm = dspy.LM(model="ollama_chat/gpt-oss:20b", api_base="http://localhost:11434", api_key="", temperature=1.0)

In [2]:
dataset = load_dataset("mlabonne/smoltldr")

examples = [
    dspy.Example({"prompt": x["prompt"], "gold_tldr": x["completion"]}).with_inputs("prompt")
    for x in dataset["train"]
]

total = len(examples)  # should be ~2000

train_end = int(0.80 * total)   # 80%
val_end   = int(0.90 * total)   # next 10%

train_set = examples[:train_end]
val_set   = examples[train_end:val_end]
test_set  = examples[val_end:]

print(len(train_set), len(val_set), len(test_set))


1600 200 200


In [3]:
# ============================================================
# 2) SIGNATURE + MODULE
# ============================================================
class GenerateTLDR(dspy.Signature):
    """
    Given a Reddit post that begins with 'POST:', generate a single-line,
    concise TL;DR (~25 words). Avoid line breaks.
    """
    prompt: str = dspy.InputField()
    tldr: str = dspy.OutputField()


class TLDRModule(dspy.Module):
    def __init__(self):
        self.generator = dspy.ChainOfThought(GenerateTLDR)

    def forward(self, prompt: str):
        out = self.generator(prompt=prompt)
        return dspy.Prediction(tldr=out.tldr)

program = TLDRModule()


In [4]:
# ============================================================
# 3) HELPERS (semantic model, post text extractor)
# ============================================================
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # avoid HF tokenizer thread issues

# Load the semantic model ONCE, globally — no lazy loading in threads
sem_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def _word_count(s: str) -> int:
    return len(s.split())

def extract_post(prompt: str):
    m = re.search(r"POST:\s*(.*?)(?:\n\s*TL;DR:|$)", prompt, flags=re.DOTALL)
    return m.group(1).strip() if m else prompt.strip()



In [5]:
# ============================================================
# 4) FEEDBACK FUNCTIONS — EXACTLY LIKE YOUR STYLE
# ============================================================

def feedback_len(pred_tldr: str, target_words=25, max_words=70):
    """
    Length feedback (normalized 0–1 score).
    """
    n = _word_count(pred_tldr)
    max_diff = max(abs(target_words - 1), abs(target_words - max_words))
    score = 1 - (abs(target_words - n) / max_diff)
    score = max(0.0, min(1.0, score))

    if score == 1.0:
        fb = f"You produced the ideal length (~{n} words)."
    elif n > target_words:
        fb = f"Your TL;DR is too long ({n} words). Aim for ~{target_words}."
    else:
        fb = f"Your TL;DR is too short ({n} words). Add key content."
    return fb, score


def feedback_style(pred_tldr: str):
    """
    Style feedback (1 if single line, else 0).
    """
    ok = ("\n" not in pred_tldr) and ("\r" not in pred_tldr)
    score = 1.0 if ok else 0.0
    if score == 1.0:
        fb = "You kept it single-line with no line breaks."
    else:
        fb = "Your TL;DR contains line breaks. It must be a single line."
    return fb, score


def feedback_sem(prompt: str, pred_tldr: str):
    """
    Semantic similarity feedback (normalized 0–1).
    """
    post = extract_post(prompt)

    # Use the globally loaded sem_model (no lazy creation in threads)
    global sem_model
    with torch.no_grad():
        e_pred = sem_model.encode(pred_tldr, convert_to_tensor=True, normalize_embeddings=True)
        e_post = sem_model.encode(post,      convert_to_tensor=True, normalize_embeddings=True)
        sim = torch.sum(e_pred * e_post).item()

    score = (sim + 1) / 2        # [-1,1] -> [0,1]
    score = max(0.0, min(1.0, score))

    if score > 0.8:
        fb = "Excellent semantic alignment — you captured the core meaning."
    elif score > 0.6:
        fb = "Moderate alignment — include 1–2 more key ideas."
    else:
        fb = "Low alignment — your TL;DR misses core points of the post."
    return fb, score

In [6]:

# ============================================================
# 5) METRIC (scalar only)
# ============================================================
def metric(example, pred, trace=None, pred_name=None, pred_trace=None):
    fb_len, s_len = feedback_len(pred.tldr)
    fb_sty, s_sty = feedback_style(pred.tldr)
    fb_sem, s_sem = feedback_sem(example["prompt"], pred.tldr)

    total = (s_len + s_sty + s_sem) / 3.0
    return total


In [7]:
# ============================================================
# BASELINE EVALUATION
# ============================================================
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=16,
    display_table=True,
    display_progress=True
)

print("\n== Baseline Evaluation ==")
evaluate(program)


== Baseline Evaluation ==
Average Metric: 181.83 / 200 (90.9%): 100%|██████████| 200/200 [00:08<00:00, 24.53it/s]

2025/11/17 11:24:42 INFO dspy.evaluate.evaluate: Average Metric: 181.8347770365852 / 200 (90.9%)


,prompt,gold_tldr,tldr,metric
0,SUBREDDIT: r/relationships TITLE: Me [16 F] with my boyfriend [18 ...,I get too worried when my bf goes out. I need help so I make it s...,16‑year‑old girl in 10‑month relationship feels anxious when boyfr...,✔️ [0.897]
1,SUBREDDIT: r/AskReddit TITLE: Indirect Demoralization at work POST...,"went up to ask for raise, got shut down before even asking, now I...",User feels demoralized after overhearing boss deny raises and beli...,✔️ [0.915]
2,SUBREDDIT: r/relationships TITLE: My boyfriend [21] hasn't made me...,"Boyf hasnt made me orgasm, I've been faking the whole time. To te...","Girl hasn't orgasmed in 5 months, faking while boyfriend notices. ...",✔️ [0.914]
3,SUBREDDIT: r/relationships TITLE: I [22F] matched with an ex [23M]...,Matched w a guy I used to date on tinder. Would like to talk to h...,I matched with ex on Tinder after 4 months apart; unsure if I shou...,✔️ [0.933]
4,SUBREDDIT: r/relationship_advice TITLE: [19/m] where do I go from ...,"talking to girl, not sure if she feels the same way as I do, but ...",He's out of a friendship with a girl and wants to know if she want...,✔️ [0.893]
...,...,...,...,...
195,SUBREDDIT: r/dating_advice TITLE: Where Does a Friend-Zone/Relatio...,! How do I know if she wants to be in the friend-zone? How do I k...,"She appears to see you as a friend; be honest about feelings, resp...",✔️ [0.911]
196,SUBREDDIT: r/AskReddit TITLE: Should I consider going back to Digg...,"Reddit is fading, especially with the lag..to the Digg user base!...","Reddit used to be fast and familiar, but now feels slower; Digg is...",✔️ [0.938]
197,SUBREDDIT: r/relationships TITLE: Boyfriend [25m] broke up with me...,Boyfriend of 3 1/2 years broke up with me and I can't move out fo...,"Lost after 3.5 years breakup, still share a home, stuck for two we...",✔️ [0.916]
198,SUBREDDIT: r/tifu TITLE: TIFU by ruining my best friends collegiat...,"Roundhoused my friend in the hip, he fell and fucked up his ankle...","After a prank, I tapped my friend, causing him to fall, break his ...",✔️ [0.919]


EvaluationResult(score=90.92, results=<list of 200 results>)

In [8]:

# ============================================================
# 6) METRIC WITH FEEDBACK (YOUR EXACT STYLE)
# ============================================================
def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    # Compute all three feedbacks and scores
    fb_len, s_len = feedback_len(pred.tldr)
    fb_sty, s_sty = feedback_style(pred.tldr)
    fb_sem, s_sem = feedback_sem(example["prompt"], pred.tldr)

    total = (s_len + s_sty + s_sem) / 3.0

    # No feedback requested -> return only scalar
    if pred_name is None:
        return total

    # GEPA-specific: return feedback for the predictor being optimized
    if pred_name == "generator.predict":
        # EXACT STYLE: individual feedback for all three
        feedback = (
            f"Length Feedback:\n{fb_len}\n\n"
            f"Style Feedback:\n{fb_sty}\n\n"
            f"Semantic Feedback:\n{fb_sem}\n\n"
            f"Overall Score: {total:.3f}\n"
            "Think step-by-step about how to produce a single-line, concise, accurate TL;DR."
        )
    else:
        feedback = "No specific predictor matched. Review reasoning carefully."

    return dspy.Prediction(score=total, feedback=feedback)

In [9]:

# ============================================================
# 7) GEPA OPTIMIZER
# ============================================================
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    num_threads=16,
    track_stats=True,
    track_best_outputs=True,
    use_merge=False,
    reflection_lm=reflection_lm
)

optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)


2025/11/17 11:24:42 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 1180 metric calls of the program. This amounts to 0.66 full evals on the train+val set.
2025/11/17 11:24:42 INFO dspy.teleprompt.gepa.gepa: Using 200 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/1180 [00:00<?, ?rollouts/s]2025/11/17 11:26:59 INFO dspy.evaluate.evaluate: Average Metric: 181.54047488838572 / 200 (90.8%)
2025/11/17 11:26:59 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.9077023744419286
GEPA Optimization:  17%|█▋        | 200/1180 [02:16<11:08,  1.47rollouts/s]2025/11/17 11:26:59 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 

Average Metric: 2.64 / 3 (87.9%): 100%|██████████| 3/3 [00:05<00:00,  1.91s/it]

2025/11/17 11:27:05 INFO dspy.evaluate.evaluate: Average Metric: 2.6367465579951253 / 3 (87.9%)


2025/11/17 11:27:11 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for generator.predict: **TL;DR Generation for Reddit Threads**

You will receive a Reddit thread excerpt that always follows this exact format:

```
SUBREDDIT: <subreddit name>
TITLE: <post title>
POST: <full body of the original Reddit post>
```

Your task is to produce a **single-line, concise TL;DR** that summarizes the entire post.  
Follow these rules:

1. **Length** – Aim for about **25 words** (±3 words).  
2. **Line breaks** – The output must be a **single line**; no newlines, bullets, or other formatting.  
3. **Content** – Capture the core narrative or conflict:
   * Key characters or actors (e.g., the poster, a partner, a friend).  
   * The main event or dilemma (e.g., breakup, accidental injury, unrequited love).  
   * Any emotional stakes or unresolved question (e.g., “he’s unsure whether to cut ties”).  
4. **What to exclude** – Omit URLs, markdown, emojis, and extraneous tags.  
5. **If 

Average Metric: 2.75 / 3 (91.7%): 100%|██████████| 3/3 [00:07<00:00,  2.35s/it]

...

2025/11/17 12:08:44 INFO dspy.evaluate.evaluate: Average Metric: 2.78872807820638 / 3 (93.0%)


2025/11/17 12:08:51 INFO dspy.teleprompt.gepa.gepa: Iteration 7: Proposed new text for generator.predict: SUBREDDIT: <subreddit name>
TITLE: <post title>
POST: <full body of the original Reddit post>
2025/11/17 12:09:05 INFO dspy.evaluate.evaluate: Average Metric: 1.5088952625239336 / 3 (50.3%)
2025/11/17 12:09:05 INFO dspy.teleprompt.gepa.gepa: Iteration 7: New subsample score 1.5088952625239336 is not better than old score 2.78872807820638, skipping
GEPA Optimization:  88%|████████▊ | 1042/1180 [44:22<06:57,  3.03s/rollouts]2025/11/17 12:09:05 INFO dspy.teleprompt.gepa.gepa: Iteration 8: Selected program 3 score: 0.9144158045835793


Average Metric: 2.77 / 3 (92.5%): 100%|██████████| 3/3 [00:08<00:00,  2.91s/it]

2025/11/17 12:09:14 INFO dspy.evaluate.evaluate: Average Metric: 2.7743208178767453 / 3 (92.5%)


2025/11/17 12:09:19 INFO dspy.teleprompt.gepa.gepa: Iteration 8: Proposed new text for generator.predict: You will receive a Reddit thread description that follows this exact format:

```
SUBREDDIT: <subreddit name>
TITLE: <title of the post>
POST: <full post text, possibly several paragraphs>
```

Your task is to generate **one single line** that is a concise TL;DR of the post.  
The TL;DR must:

1. **Capture the core meaning** of the post, including the key decision point, conflict, or question the poster is asking for help with.  
2. **Be approximately 20‑30 words** (aim for ~25 words). If the generated line is shorter than 20 words or longer than 30 words, adjust by adding or removing **one descriptive word**.  
3. **Be a single line** – do **not** insert line breaks, bullet points, or any additional formatting.  
4. **Contain no extra commentary** – do not add your own thoughts, labels, or instructions.  
5. **Read as a standalone summary** that a reader can understand without see

In [10]:
# ============================================================
# 8) SEE OPTIMIZED PROMPTS
# ============================================================
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")


Predictor: generator.predict
Prompt:
**TL;DR Generation for Reddit Threads**

You will receive a Reddit thread excerpt that always follows this exact format:

```
SUBREDDIT: <subreddit name>
TITLE: <post title>
POST: <full body of the original Reddit post>
```

Your task is to produce a **single‑line TL;DR** that summarizes the entire post. Follow these rules:

1. **Length** – The TL;DR must contain **exactly 25 words** (no more, no less).  
2. **Line breaks** – The output must be a **single line**; no newlines, bullets, or other formatting.  
3. **Content** – Capture the core narrative or conflict:
   * Main actors (e.g., poster, partner, friend).  
   * Central event or dilemma (e.g., health issue, breakup, sabotage).  
   * Emotional stakes or unresolved question (e.g., “is the medication the cause?”).  
4. **What to exclude** – Omit URLs, markdown, emojis, extraneous tags, and any filler words that do not add meaning.  
5. **If the post is extremely short** – Use the entire content

In [11]:
# ============================================================
# 9) FINAL POST-GEPA EVALUATION
# ============================================================
print("\n== GEPA-Optimized Evaluation ==")
evaluate(optimized_program)


== GEPA-Optimized Evaluation ==
Average Metric: 186.43 / 200 (93.2%): 100%|██████████| 200/200 [12:13<00:00,  3.67s/it]

2025/11/17 12:29:59 INFO dspy.evaluate.evaluate: Average Metric: 186.425146161075 / 200 (93.2%)


,prompt,gold_tldr,tldr,metric
0,SUBREDDIT: r/relationships TITLE: Me [16 F] with my boyfriend [18 ...,I get too worried when my bf goes out. I need help so I make it s...,16‑year‑old girlfriend feels overly attached to 18‑year‑old boyfri...,✔️ [0.890]
1,SUBREDDIT: r/AskReddit TITLE: Indirect Demoralization at work POST...,"went up to ask for raise, got shut down before even asking, now I...","Year without raise and duties, I planned to ask, but boss’s dispar...",✔️ [0.916]
2,SUBREDDIT: r/relationships TITLE: My boyfriend [21] hasn't made me...,"Boyf hasnt made me orgasm, I've been faking the whole time. To te...",18‑year‑old woman never orgasmed with 21‑year‑old boyfriend after ...,✔️ [0.924]
3,SUBREDDIT: r/relationships TITLE: I [22F] matched with an ex [23M]...,Matched w a guy I used to date on tinder. Would like to talk to h...,"22‑year‑old woman, previously dating ex‑boyfriend, unexpectedly ma...",✔️ [0.934]
4,SUBREDDIT: r/relationship_advice TITLE: [19/m] where do I go from ...,"talking to girl, not sure if she feels the same way as I do, but ...",19‑year‑old male wonders if 18‑year‑old female friend wants a roma...,✔️ [0.915]
...,...,...,...,...
195,SUBREDDIT: r/dating_advice TITLE: Where Does a Friend-Zone/Relatio...,! How do I know if she wants to be in the friend-zone? How do I k...,"Seventeen‑year‑old poster wonders if his 16‑year‑old friend, with ...",✔️ [0.911]
196,SUBREDDIT: r/AskReddit TITLE: Should I consider going back to Digg...,"Reddit is fading, especially with the lag..to the Digg user base!...","I loved Reddit’s fast, well‑organized front page like Digg’s, but ...",✔️ [0.968]
197,SUBREDDIT: r/relationships TITLE: Boyfriend [25m] broke up with me...,Boyfriend of 3 1/2 years broke up with me and I can't move out fo...,"I broke up with my 25M boyfriend; we live together, I must continu...",✔️ [0.908]
198,SUBREDDIT: r/tifu TITLE: TIFU by ruining my best friends collegiat...,"Roundhoused my friend in the hip, he fell and fucked up his ankle...","I attempted to tap my friend Ryan, causing him to fall, break his ...",✔️ [0.939]


EvaluationResult(score=93.21, results=<list of 200 results>)

## Performance Improvement Summary

Display the accuracy lift from baseline (90.92%) to optimized (93.21%).

### Comparison: GEPA vs MIPROv2

| Optimizer | Baseline | Optimized | Improvement |
|-----------|----------|-----------|-------------|
| GEPA | 90.92% | 93.21% | +2.29 pp |
| MIPROv2 | 90.92% | 91.13% | +0.21 pp |

*pp = percentage points*  
Corresponding MIPROv2 results are documented in `miprov2_3_tldr.ipynb`. 
